In [ ]:
import pyautogui  
import time
import random
import os
from PIL import ImageGrab
from pynput.keyboard import Listener as KeyboardListener, Key

# Variável global para controlar a execução do código
rodando = True

# Função para parar a execução ao pressionar ESC
def on_press(key):
    global rodando
    if key == Key.esc:
        print("Execução interrompida pelo usuário (ESC).")
        rodando = False
        return False  # Para o listener de teclado

# Função para mover a janela do jogo
def mover_janela(jogo, x, y):
    pyautogui.getWindowsWithTitle(jogo)[0].moveTo(x, y)

# Função para capturar prints
def capturar_print(nome_arquivo, coordenadas):
    if not os.path.exists("./prints"):
        os.makedirs("./prints")
    imagem = ImageGrab.grab(bbox=coordenadas)
    imagem.save(f"./prints/{nome_arquivo}.png")

# Função para escolher uma expressão facial e movimento de boca com base na emoção
def selecionar_face_mouth(emocao):
    faces = {
        "feliz": [(1501, 330), (1576, 548), (1420, 277), (1511, 376), (1588, 379)],
        "triste": [(1502, 539), (1415, 596), (1498, 540)],
        "neutro": [(1428, 433), (1418, 478), (1581, 437), (1493, 275), (1506, 273), (1566, 280)],
        "surpreso": [(1427, 276), (1422, 331), (1424, 588), (1423, 545), (1580, 489)]
    }
    
    mouths = {
        "feliz": [(1422, 278), (1577, 283), (1419, 330), (1579, 433), (1584, 485), (1579, 537), (1496, 594)],
        "triste": [(1502, 279), (1500, 328), (1416, 492), (1500, 441), (1569, 641)],
        "neutro": [(1504, 543), (1588, 330)],
        "boca_aberta_feliz": [(1584, 279), (1498, 333), (1414, 388), (1589, 374), (1422, 486), (1588, 486), (1428, 573)],
        "boca_aberta_surpreso": [(1586, 331), (1405, 325), (1583, 424)]
    }
    
    # Se a emoção for 'surpreso', usar 'boca_aberta_surpreso' para escolher a boca
    if emocao == "surpreso":
        mouth_coord = random.choice(mouths["boca_aberta_surpreso"])
    else:
        mouth_coord = random.choice(mouths[emocao])

    face_coord = random.choice(faces[emocao])
    
    return face_coord, mouth_coord

# Função para processar cada personagem
def processar_personagem(personagem, coordenada_personagem, fala_emocao):
    print(f"Processando personagem: {personagem}")
    
    # Clicar no personagem
    pyautogui.click(coordenada_personagem)
    time.sleep(1)
    
    fala, emocao = fala_emocao
    
    # Clicar no chat para iniciar a seleção da face e boca
    pyautogui.click(1591, 351)  # Coordenada do chat
    time.sleep(1)
    
    # Selecionar face e boca
    face_coord, mouth_coord = selecionar_face_mouth(emocao)
    
    pyautogui.click(face_coord)  # Clicar na face
    time.sleep(1)
    
    pyautogui.click(1492, 653)  # Clicar no botão de seleção da boca
    time.sleep(0.5)
    pyautogui.click(mouth_coord)  # Clicar na boca
    time.sleep(1)
    
    # Clicar duas vezes na área para confirmar (1573, 217)
    print("Clicando duas vezes na área (1573, 217)...")
    pyautogui.click(1573, 217)  # Primeiro clique
    time.sleep(0.5)  # Pequena pausa antes do segundo clique
    pyautogui.click(1573, 217)  # Segundo clique
    time.sleep(1)

    # Clicar no chat para escrever a fala
    pyautogui.click(1595, 288)  # Coordenada do chat
    time.sleep(1)
    
    # Escrever a fala no chat
    print(f"Escrevendo a fala: {fala}")
    pyautogui.write(fala)
    time.sleep(1)
    
    # Fechar o chat
    pyautogui.click(1437, 398)  # Coordenada para fechar o chat
    time.sleep(1)

# Função para clicar no botão "view" para avançar
def clicar_view():
    print("Clicando no botão 'view'...")
    pyautogui.click(859, 631)  # Coordenada do botão "view"
    time.sleep(2)  # Aguardar a ação ser concluída

# Função principal para processar todos os personagens
def processar_falas(falas_emocoes):
    personagens = [
        {"nome": "Itadori", "coordenada": (1659, 210)},
        {"nome": "Nobara", "coordenada": (1659, 259)},
        {"nome": "Megumi", "coordenada": (1661, 306)},
        {"nome": "Maki", "coordenada": (1660, 355)},
        {"nome": "Toge", "coordenada": (1657, 403)},
        {"nome": "Gojo", "coordenada": (1657, 450)}
    ]

    total_prints = 0
    max_prints = 300
    
    while total_prints < max_prints:
        for i, fala_emocao in enumerate(falas_emocoes):
            if not rodando:  # Checa se a execução foi interrompida
                break
            personagem = personagens[i % len(personagens)]
            
            # Processar personagem atual (sem capturar o print do personagem)
            processar_personagem(personagem["nome"], personagem["coordenada"], fala_emocao)
            
            # Clicar no botão "view"
            clicar_view()
            
            # Capturar o print da área "view" logo após clicar
            capturar_print(f"print_view_{total_prints + 1}", (809, 184, 1703, 675))
            time.sleep(1)
            total_prints += 1
            
            if total_prints >= max_prints:
                break  # Se atingiu o número de prints, parar o loop

            # Verifica se existe um próximo personagem
            if i + 1 < len(personagens):
                prox_personagem = personagens[i + 1]
                print(f"Clicando três vezes no próximo personagem: {prox_personagem['nome']}")
                
                # Clicar três vezes no próximo personagem
                for _ in range(3):
                    pyautogui.click(prox_personagem["coordenada"])
                    time.sleep(0.5)  # Pequena pausa entre os cliques

# Ler falas do arquivo txt
def ler_falas(arquivo):
    falas_emocoes = []
    with open(arquivo, "r") as f:
        for linha in f:
            fala, emocao = linha.rsplit("(", 1)
            emocao = emocao.replace(")", "").replace("\"", "").strip()
            falas_emocoes.append((fala.strip(), emocao))
    return falas_emocoes

# Função principal
def main():
    # Iniciar o listener do teclado para ESC
    with KeyboardListener(on_press=on_press) as listener:
        mover_janela("Gacha Club", 800, 150)
        time.sleep(2)

        pyautogui.click(1390, 1060)
        time.sleep(2)

        falas_emocoes = ler_falas("falas.txt")
        processar_falas(falas_emocoes)

        listener.join()  # Espera o listener de teclado encerrar

if __name__ == "__main__":
    main()


Processando personagem: Itadori
Clicando duas vezes na área (1573, 217)...
Escrevendo a fala: Itadori: "Estou muito animado hoje!
Clicando no botão 'view'...
Clicando três vezes no próximo personagem: Nobara
Processando personagem: Nobara
Clicando duas vezes na área (1573, 217)...
Escrevendo a fala: Nobara: "Isso eh decepcionante...
Clicando no botão 'view'...
Clicando três vezes no próximo personagem: Megumi
Processando personagem: Megumi
Clicando duas vezes na área (1573, 217)...
Escrevendo a fala: Megumi:"Nem bom, nem ruim.
Clicando no botão 'view'...
Clicando três vezes no próximo personagem: Maki
Processando personagem: Maki
Clicando duas vezes na área (1573, 217)...
Escrevendo a fala: Maki: "Uau! Isso foi incrivel!
Clicando no botão 'view'...
Clicando três vezes no próximo personagem: Toge
Processando personagem: Toge
Clicando duas vezes na área (1573, 217)...
Escrevendo a fala: Toge: "oh mds
Execução interrompida pelo usuário (ESC).
Clicando no botão 'view'...
Clicando três veze